# Hakha Chin Whisper Fine-Tuning (Colab)

This notebook is configured to run entirely on Google Colab. It mounts Google Drive, installs dependencies, prepares datasets, fine-tunes Whisper on Hakha Chin, and saves outputs back to Drive.

Requirements:
- Google Colab GPU runtime preferred
- Your project folder in Drive: /content/drive/MyDrive/ChinTranslator
- Expected files (if using aligned segments): aligned_train_data.json, aligned_val_data.json


In [ ]:
# Detect Colab Runtime
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

import os
from pathlib import Path

if not IN_COLAB:
    print("⚠️ Not running in Google Colab. This notebook is intended for Colab execution.")
else:
    print("✅ Running in Google Colab.")


In [ ]:
# Verify and Configure Accelerator
import subprocess, sys

try:
    import torch
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")
    if device == 'cuda':
        try:
            # Show GPU info
            print(subprocess.run(['nvidia-smi'], check=False, capture_output=True, text=True).stdout)
            print(f"Torch CUDA: {torch.version.cuda}")
        except Exception:
            pass
    else:
        print("⚠️ GPU not detected. Training will be slow on CPU.")
except Exception as e:
    print("PyTorch not yet installed; will install in the next step.")


In [ ]:
# Install Dependencies
%pip -q install transformers datasets accelerate evaluate jiwer librosa soundfile sentencepiece torchaudio tensorboard

In [ ]:
# Authenticate and Mount Google Drive
if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
else:
    print("Skipping Drive mount (not in Colab)")


In [ ]:
# Clone or Update Repository from GitHub (optional)
import os, subprocess
REPO_URL = 'https://github.com/trinitron83/ChinTranslator2'
WORK_DIR = '/content/ChinTranslator2'

if IN_COLAB:
    if not os.path.exists(WORK_DIR):
        !git clone $REPO_URL $WORK_DIR
    else:
        %cd $WORK_DIR
        !git pull --rebase
    %cd /content
else:
    print("Skipping git clone/update (not in Colab)")


In [ ]:
# Set Project Paths and Environment Variables
import sys
ROOT_DIR = '/content/drive/MyDrive/ChinTranslator'
DATA_DIR = f'{ROOT_DIR}'
OUTPUT_DIR = f'{ROOT_DIR}/whisper-hakha-chin-v3'

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = '1'

# Ensure directories exist
os.makedirs(ROOT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Add repo to Python path if cloned
if IN_COLAB and os.path.exists('/content/ChinTranslator2'):
    sys.path.append('/content/ChinTranslator2')

print('ROOT_DIR:', ROOT_DIR)
print('OUTPUT_DIR:', OUTPUT_DIR)


In [ ]:
# Load Secrets and Config (optional)
from getpass import getpass

# Example: Hugging Face token (not required for public models)
HF_TOKEN = os.environ.get('HF_TOKEN', '')
if not HF_TOKEN:
    print('No HF token set; proceeding with public access.')


In [ ]:
# Run Unit Tests with Pytest (optional)
if IN_COLAB and os.path.exists('/content/ChinTranslator2/tests'):
    %cd /content/ChinTranslator2
    !pytest -q || true
    %cd /content
else:
    print('No tests found or not in Colab; skipping pytest.')


In [ ]:
# Download or Cache Data (verify presence of aligned files)
from pathlib import Path
import json

aligned_train = Path(f"{ROOT_DIR}/aligned_train_data.json")
aligned_val = Path(f"{ROOT_DIR}/aligned_val_data.json")
training_full = Path(f"{ROOT_DIR}/training_data.json")

print('Looking for:', aligned_train, aligned_val)
print('Exists:', aligned_train.exists(), aligned_val.exists())

# No external download needed; files are expected in Drive.


In [ ]:
# Execute Project Entry Points: Prepare datasets
from datasets import Dataset, Audio

# Load data from JSON files
if aligned_train.exists() and aligned_val.exists():
    with open(aligned_train, 'r', encoding='utf-8') as f:
        train_data = json.load(f)
    with open(aligned_val, 'r', encoding='utf-8') as f:
        val_data = json.load(f)
    print(f"Loaded aligned segments: train={len(train_data)}, val={len(val_data)}")
elif training_full.exists():
    # Fallback: split full training_data.json 80/20
    with open(training_full, 'r', encoding='utf-8') as f:
        data = json.load(f)
    n = len(data)
    split = int(0.8 * n)
    train_data = data[:split]
    val_data = data[split:]
    print(f"Loaded full dataset and split: train={len(train_data)}, val={len(val_data)}")
else:
    raise FileNotFoundError("No aligned or full training data JSON files found in Drive.")

# Normalize audio paths
from pathlib import PurePosixPath

missing = 0
for lst in (train_data, val_data):
    for item in lst:
        p = item['audio']
        # If path is relative, prefix with ROOT_DIR
        if not p.startswith('/'):
            p = f"{ROOT_DIR}/{p}"
        # If path points to local repo-like path, ensure it uses ROOT_DIR
        if '/ChinTranslator/' in p and not p.startswith('/content/drive/'):
            # Replace up to 'ChinTranslator' with ROOT_DIR
            idx = p.rfind('/ChinTranslator')
            p = f"{ROOT_DIR}{p[idx+len('/ChinTranslator'):] }"
        item['audio'] = p
        if not os.path.exists(p):
            missing += 1

print(f"Missing audio files (will be skipped at load time): {missing}")

# Build HF Datasets
train_dataset = Dataset.from_dict({
    'audio': [x['audio'] for x in train_data],
    'text': [x['text'] for x in train_data],
})
val_dataset = Dataset.from_dict({
    'audio': [x['audio'] for x in val_data],
    'text': [x['text'] for x in val_data],
})

train_dataset = train_dataset.cast_column('audio', Audio(sampling_rate=16000))
val_dataset = val_dataset.cast_column('audio', Audio(sampling_rate=16000))

print(train_dataset)
print(val_dataset)


In [ ]:
# Load Whisper model and processor
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
)

MODEL_NAME = 'openai/whisper-small'
TASK = 'transcribe'

from pathlib import Path
prev_model_dir = Path(f"{ROOT_DIR}/whisper-hakha-chin-v2")
if prev_model_dir.exists():
    print(f"Continuing training from: {prev_model_dir}")
    processor = WhisperProcessor.from_pretrained(str(prev_model_dir), language=None, task=TASK)
    model = WhisperForConditionalGeneration.from_pretrained(str(prev_model_dir))
else:
    print(f"Loading base model: {MODEL_NAME}")
    processor = WhisperProcessor.from_pretrained(MODEL_NAME, language=None, task=TASK)
    model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)

# Recommended project settings
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False  # disable for training

# Explicitly disable gradient checkpointing due to prior issues in project notes
try:
    model.gradient_checkpointing_disable()
except Exception:
    pass

print('✅ Model and processor ready')


In [ ]:
# Prepare data mapping

def prepare_data(batch):
    audio = batch['audio']
    out = processor.feature_extractor(
        audio['array'], sampling_rate=audio['sampling_rate']
    )
    batch['input_features'] = out.input_features[0]
    batch['labels'] = processor.tokenizer(batch['text']).input_ids
    return batch

print('Mapping datasets...')
train_dataset_proc = train_dataset.map(prepare_data, remove_columns=train_dataset.column_names)
val_dataset_proc = val_dataset.map(prepare_data, remove_columns=val_dataset.column_names)
print('✅ Datasets processed')


In [ ]:
# Data collator
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if labels.shape[1] > 0 and (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
print('✅ Collator ready')


In [ ]:
# Metrics
import evaluate
metric = evaluate.load('wer')

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {'wer': wer}

print('✅ Metrics ready')


In [ ]:
# Training arguments
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=5e-6,
    warmup_steps=50,
    num_train_epochs=15,
    gradient_checkpointing=False,
    fp16=True if (torch.cuda.is_available()) else False,
    evaluation_strategy='steps',
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=200,
    eval_steps=200,
    logging_steps=50,
    report_to=['tensorboard'],
    load_best_model_at_end=True,
    metric_for_best_model='wer',
    greater_is_better=False,
)

print(training_args)


In [ ]:
# Initialize Trainer
from transformers import Seq2SeqTrainer, TrainerCallback

class ProgressCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"Step {state.global_step}: {logs}")

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset_proc,
    eval_dataset=val_dataset_proc,
    data_collator=collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[ProgressCallback()],
)

print('✅ Trainer ready')


In [ ]:
# Start Training
print('\n🚀 Starting training...')
train_result = trainer.train()
print('\n✅ Training complete!')
train_result


In [ ]:
# Persist Artifacts to Drive
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)
print(f"Saved model and processor to: {OUTPUT_DIR}")


In [ ]:
# Quick Inference Test
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import librosa

print('Loading model for inference...')
proc = WhisperProcessor.from_pretrained(OUTPUT_DIR)
mdl = WhisperForConditionalGeneration.from_pretrained(OUTPUT_DIR)
mdl.to('cuda' if torch.cuda.is_available() else 'cpu')

sample_audio = train_data[0]['audio'] if len(train_data) else None
print('Sample:', sample_audio)
if sample_audio and os.path.exists(sample_audio):
    audio, _ = librosa.load(sample_audio, sr=16000)
    inputs = proc(audio, sampling_rate=16000, return_tensors='pt').input_features
    inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
    gen_ids = mdl.generate(inputs, max_length=200, repetition_penalty=1.5, no_repeat_ngram_size=3)
    text = proc.batch_decode(gen_ids, skip_special_tokens=True)[0]
    print('Transcription:\n', text)
else:
    print('No valid sample audio found for quick test.')


# Next steps
- Ensure aligned JSONs are in `/content/drive/MyDrive/ChinTranslator`.
- Consider continuing from `whisper-hakha-chin-v2` if present.
- Avoid enabling gradient checkpointing due to prior PyTorch issues.
- Prefer file uploads over mic recordings unless you normalize audio to 16kHz mono.
- Scale data (more chapters, diverse speakers) for better generalization.
